#KNU 한국어 감정사전 업로드
(출처:http://dilab.kunsan.ac.kr/knusl.html)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# KNU 감정 사전
positive_word = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/pos_pol_word.txt")
negative_word = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/neg_pol_word.txt")

Positive = positive_word.transpose() # list(positive_word['word'])
Negative = negative_word.transpose() # list(negative_word['word'])

print(Positive)
print(Negative)

     0     1      2     3      4      5     ... 4862 4863   4864 4865  4866  4867
word  (-;  (^^)  (^-^)  (^^*  (^_^)  (^o^)  ...  상위권  안정감  적극적이다   승리  승리하다  유명하다

[1 rows x 4868 columns]
     0     1     2       3     4    5     ... 9821 9822 9823   9824 9825 9826
word   가난  가난뱅이  가난살이  가난살이하다  가난설음  가난에  ...   갈등  비리를   의혹  내팽개치다   횡령  불안증

[1 rows x 9827 columns]


In [ ]:
# 부정
cols_n = []
for i in range(0, 9826): # 감정사전 내 부정어의 개수
  cols_n.append(i)
#print(cols_n)
Negative["Keyword"] = Negative[cols_n].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
Negative

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9788,9789,9790,9791,9792,9793,9794,9795,9796,9797,9798,9799,9800,9801,9802,9803,9804,9805,9806,9807,9808,9809,9810,9811,9812,9813,9814,9815,9816,9817,9818,9819,9820,9821,9822,9823,9824,9825,9826,Keyword
word,가난,가난뱅이,가난살이,가난살이하다,가난설음,가난에,가난에 쪼들려서,가난하게,가난하고,가난하고 어렵다,가난하고 천한,가난하다,가난하여,가난한,가난한 데다가,가난한 사람을,가난히,가년스럽다,가녕스럽다,가누지 못하고,가누지 못하다,가누지 못할,가누지 못할 정도로,가누지 못함,가눌 수 없게,가눌 수 없을,가눌 수 없이,가당찮다,가당찮이,가두거나 해치거나,가망이 없게,가슴 아파하다,가슴 아파함,가슴 아프게,가슴 아프고,가슴 아프다,가슴속에 근심이나,가슴앓이,가슴앓이하다,가슴이 갑갑할,...,심통,심통부리다,가련하다,고독하다,고독한,곤욕,난해하다,뼈저리다,쓴웃음,씁쓸하다,아휴,울컥하다,으앙,음울하다,적적하다,청승맞다,청승궂다,청승맞게,흑흑,고리타분하다,신통찮다,심드렁하다,꺼림찍하다,찜찜하다,달갑잖다,못미덥다,콧웃음치다,감질나다,아리송하다,알쏭하다,알쏭달쏭하다,실점,오류,갈등,비리를,의혹,내팽개치다,횡령,불안증,가난 가난뱅이 가난살이 가난살이하다 가난설음 가난에 가난에 쪼들려서 가난하게 가난하...


In [ ]:
# 긍정
cols_n = []
for i in range(0, 4867): # 감정사전 내 긍정어의 개수
  cols_n.append(i)
#print(cols_n)
Positive["Keyword"] = Positive[cols_n].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
Positive

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4829,4830,4831,4832,4833,4834,4835,4836,4837,4838,4839,4840,4841,4842,4843,4844,4845,4846,4847,4848,4849,4850,4851,4852,4853,4854,4855,4856,4857,4858,4859,4860,4861,4862,4863,4864,4865,4866,4867,Keyword
word,(-;,(^^),(^-^),(^^*,(^_^),(^o^),*^^*,/^o^\,:(,:'-(,:-(,:-),:|,;,;),;-),^///^,^^,^^//,<3,=),XD,ㄱㅅ,ㄱㅇㄷ,가격이 싸다,가까이 사귀어,가까이하다,가다듬어,가다듬어 수습하는,가다듬어 수습하다,가다듬어 정하다,가당히,가뜬하게,가라앉혀 바로잡다,가르치며,가르침,가벼운 마음으로,가볍게 웃다,가볍고,가볍고 보드랍게,...,흥이 나서,흥취,흥취 있게,흥하다,흥행,흥행되다,흥행작,흥행하다,흥행화하다,희락,희희낙락,희희낙락거리다,희희낙락대다,희희낙락하다,희희호호,힘이 강하다,힘이 기운차게,힘차게,힘차게 진행되는,힘차고,힘차고 대단한,힘차고 튼튼하다,힘차다,힘찬,힘찬 기운이,섹시하다,원더풀,쾌적,쾌적한,쾌적하다,함박웃음,득점,상위,상위권,안정감,적극적이다,승리,승리하다,유명하다,(-; (^^) (^-^) (^^* (^_^) (^o^) *^^* /^o^\ :( ...


# 종목별 ESG 기사 및 불용어 업로드

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# 종목명 분리
list_1 = pd.read_csv("/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv")
list_2 = list_1.loc[0:99] # 시총 상위 100개 기업
list_3 = list_2[['종목명']]
print(list_3)

         종목명
0       삼성전자
1     SK하이닉스
2      NAVER
3   삼성바이오로직스
4      삼성전자우
..       ...
95      신풍제약
96    현대차2우B
97     GS리테일
98   일진머티리얼즈
99     SK케미칼

[100 rows x 1 columns]


In [ ]:
# E부문 '친환경' 크롤링 기사 로드 
df_E1 = pd.read_csv('/content/drive/MyDrive/미래에셋/ESG 키워드별 크롤링/data/E부문_상위100_친환경.csv')
#NULL값 확인
#df_E1['Article'].isnull().sum()



# 불용어 리스트 로드
nltk.download('punkt')
stopwords_list_full = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/한국어불용어100.txt", sep = "\t", names = ['Word', 'Code', 'Freq'])
stopwords_list = stopwords_list_full['Word']
stopwords = stopwords_list.values.tolist()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


불용어 단어 리스트 출처: https://bab2min.tistory.com/544

# 종목별 기사본문 - 긍정/부정어 사전에 대한 코사인 유사도 평균 산출

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from numpy import dot
from numpy.linalg import norm
import re

In [ ]:
result = [] # 모든 종목에 대한 데이터프레임 데이터


for stockName in list_3['종목명']:
  df_E1_stock = (df_E1['stockName'] == stockName)
  df_E1_per = df_E1[df_E1_stock]
  df_E1_per = df_E1_per.reset_index(drop=True, inplace=False)
  #print(df_E1_per) 


  # 기사 본문에 대한 특수문자 제거
  def sub_special(sentence):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', "", sentence)
  df_E1_Article = df_E1_per['Article'].apply(sub_special)
  cnt = int(len(df_E1_Article.index) * 20.4) # 해당 기간 동안의 기사 개수
  #print(cnt)


  # 기사 본문에 대한 불용어 제거 
  for i, document in enumerate(df_E1_Article):
      clean_words = [] 
      for word in nltk.tokenize.word_tokenize(document): 
          if word not in stopwords: #불용어 제거
              clean_words.append(word)
      #print("불용어 제거 완료")
      #print(i)
      #print(clean_words)     
      df_E1_Article[i] = ' '.join(clean_words)


  # 최종 기사 데이터프레임
  df_E1_Article = pd.DataFrame(df_E1_Article, columns=['Article'])
  #print(df_E1_Article)
  

  # 기사 본문에 대한 tf-idf 계산
  vectorizer = TfidfVectorizer()
  tfidf_article_matrix = vectorizer.fit_transform(df_E1_Article['Article'])
  #print(tfidf_article_matrix.shape)
  

  # 부정어에 대한 tf-idf 계산
  tfidf_negative_matrix = vectorizer.transform([Negative.at['word','Keyword']])
  #print(tfidf_negative_matrix.shape)

  # 긍정어에 대한 tf-idf 계산
  tfidf_positive_matrix = vectorizer.transform([Positive.at['word','Keyword']])
  #print(tfidf_positive_matrix.shape)

  # 각각의 감정사전과 기사 본문에 대한 코사인 유사도 산출
  #print("기사본문 - 부정어 간 코사인 유사도 평균")
  cos_sim_n = linear_kernel(tfidf_article_matrix, tfidf_negative_matrix)
  #print("기사본문 - 긍정어 간 코사인 유사도 평균")
  cos_sim_p = linear_kernel(tfidf_article_matrix, tfidf_positive_matrix)
  
  result.append([cnt, np.mean(cos_sim_n),np.mean(cos_sim_p)]) # 관련 기사 개수, 부정어 유사도, 긍정어 유사도


In [ ]:
# 모든 종목에 대한 최종 데이터프레임
df_result = pd.DataFrame(data = result, columns = ['Article_cnt', 'Negative_sim', 'Positive_sim'])
df_result_fin = list_3.join(df_result, how='left')
df_result_fin = df_result_fin.rename(columns = {'종목명': 'Stock'})

print(df_result_fin)

       Stock  Article_cnt  Negative_sim  Positive_sim
0       삼성전자          652      0.014603      0.017573
1     SK하이닉스          530      0.015041      0.025192
2      NAVER           81      0.039882      0.124230
3   삼성바이오로직스          428      0.018144      0.022012
4      삼성전자우           61      0.088789      0.105537
..       ...          ...           ...           ...
95      신풍제약          122      0.037327      0.053597
96    현대차2우B           40      0.095644      0.120896
97     GS리테일          285      0.023181      0.045082
98   일진머티리얼즈          285      0.021537      0.027145
99     SK케미칼          428      0.018862      0.026390

[100 rows x 4 columns]


# 100개 종목에 대한 최종 점수 및 랭킹

In [ ]:
# 종목별 점수 매기기

# 부정 유사도 > 긍정 유사도인 경우 확인
for i in range (0, 100):
  
  if df_result_fin.loc[i, 'Negative_sim'] > df_result_fin.loc[i, 'Positive_sim']: # 긍정 < 부정일 경우, 
    df_result_fin.loc[i,'Score'] = - df_result_fin.loc[i,'Article_cnt'] * (1 + (df_result_fin.loc[i,'Negative_sim'] - df_result_fin.loc[i,'Positive_sim'] ))


  else:  # 부정 < 긍정일 경우, 아티클 개수 * (1 + (긍정어 유사도 - 부정어 유사도))
    df_result_fin.at[i,'Score'] = df_result_fin.at[i,'Article_cnt'] * (1 + (df_result_fin.at[i,'Positive_sim'] - df_result_fin.at[i,'Negative_sim'] ))



# 종목별 랭킹 나열
df_result_fin = df_result_fin.sort_values(by=['Score'], axis=0, ascending = False)
df_result_fin = df_result_fin.reset_index (drop = True)

for i in range (0, 100):
  df_result_fin.at[i, 'Ranking'] = int(i+1)

print(df_result_fin)
df_result_fin.to_csv("E기사_친환경.csv", header = True, index = False)

    index    Stock  Article_cnt  ...  Positive_sim       Score  Ranking
0      47   한국조선해양          816  ...      0.018112  819.524064      1.0
1      23       SK          816  ...      0.015263  818.984692      2.0
2       8      현대차          795  ...      0.020101  800.176552      3.0
3      27       LG          693  ...      0.026304  700.916262      4.0
4      86       GS          652  ...      0.031333  660.498660      5.0
..    ...      ...          ...  ...           ...         ...      ...
95     66  메리츠금융지주           81  ...      0.055183   82.721861     96.0
96     32  삼성에스디에스           81  ...      0.050427   81.243404     97.0
97      4    삼성전자우           61  ...      0.105537   62.021654     98.0
98     96   현대차2우B           40  ...      0.120896   41.010084     99.0
99     84    메리츠화재          204  ...      0.043011 -206.392623    100.0

[100 rows x 7 columns]
